# **_<div class="alert alert-info"><font color='darkred'> Cartografiando las Lomas</font></div>_**

## 1. Importación de las bibliotecas necesarias

In [23]:
import ee
# Autentificar
ee.Authenticate()
# Inicializamos
ee.Initialize()

Enter verification code: 4/1AY0e-g5bl-AwBoWU06rKh-zF1BKsQfPDYxQj5lmNWYcd5O8QlrUsoFTubmQ

Successfully saved authorization token.


In [24]:
import geemap as emap
Map = emap.Map()
Map.add_basemap('ROADMAP')

## 2. Visualización y descarga de índices NDVI MODIS para la provincia de Lima

In [25]:
# Establecemos la region de mapeo (provincia de Lima)
region = ee.FeatureCollection('users/geoyons/tugee/LimaProv')

# Definimos la coleccion
col = 'MODIS/006/MOD13Q1'

# Definimos las bandas
bands = ['NDVI']

# Funcion de filtro de pixeles contaminados usando la banda de fiabilidad
def maskSQA(image):
    mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
    imageMask = image.updateMask(mask)
    return imageMask
    
# Filtrado de la coleccion MODIS
colMODIS = ee.ImageCollection(col)\
        .filterDate('2018-01-01', '2018-12-31')\
        .filterBounds(region)\
        .map(maskSQA)\
        .reduce(ee.Reducer.max()).divide(10000).clip(region)\
        .reproject(crs = 'EPSG:4326', scale = 250)

# Seleccion del indice NDVI
ndvi = colMODIS.select([0], bands)

# Definimos los parametros de visualizacion
visParams = {
  'min': 0.02,
  'max': 0.4,
  'palette': ['FFFFFF','CC9966','CC9900','996600', '33CC00', '009900','006600']
}

# Visualizamos el ndvi y la region de estudio
Map.centerObject(region, 9)
Map.addLayer(region, {}, 'Provincia de Lima', True)
Map.addLayer(ndvi, visParams, 'NDVImax MODIS 2018')
Map.add_colorbar(visParams, label="Normalized Difference Vegetation Index (NDVI)")
Map

Map(center=[-12.020761229806144, -76.91462520735031], controls=(WidgetControl(options=['position'], widget=HBo…